In [1]:
import pandas as pd 
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
df = pd.read_excel('Online Retail.xlsx')
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [3]:
# 1) Inicialmente, faça uma preparação dos dados, sendo:

# a) Remover espaços em branco do começo e fim da Descrição (str.strip())
df['Description'] = df['Description'].str.strip()

# b) Remover linhas que não possuem InvoiceNo
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)

# c) Converter InvoiceNo para string
df['InvoiceNo'] = df['InvoiceNo'].astype('str')

In [4]:
# 2) Converter os dados para o formato de transações, já filtrando apenas para o país "France":

cesta_franca = (
    df[df['Country'] == 'France']
    .groupby(['InvoiceNo', 'Description'])['Quantity']
    .sum().unstack().fillna(0).reset_index()
    .set_index('InvoiceNo')
)
cesta_franca

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C579532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C579562,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C580161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# 3) Formatar a cesta, alterando valores maiores que 1 (que representam a
# quantidade) em 1, e valores menores que 0 (caso ocorram) em 0.
# Também devemos remover a coluna ‘POSTAGE’ que representa o
# envio, e não um produto.

def encode_units(x):
    if x <= 0:
        return 0
    else: 
        return 1
cesta_franca = cesta_franca.applymap(encode_units)
cesta_franca.drop('POSTAGE', inplace = True, axis = 1)
cesta_franca 

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C579532,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C579562,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C580161,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# 4) Calcule os itemsets frequentes e as regras de associação, e responda às
# seguintes perguntas (em relação à compras feitas na França apenas)

# a) Qual a regra com maior suporte?

# Usar o max para pegar o maior suporte
# min_support nessa questão tem que ser 0,02?????
freq_franca = apriori(cesta_franca, min_support = 0.02, use_colnames = True)
rules_franca = association_rules(freq_franca, metric='support', min_threshold=0.02)
rules_franca[rules_franca['support'] == rules_franca['support'].max()]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
770,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.108460,0.117137,0.104121,0.960000,8.195556,0.091417,22.071584
771,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.117137,0.108460,0.104121,0.888889,8.195556,0.091417,8.023861


In [8]:
# b) Qual a regra com maior confiança?

# Usar o max para pegar a maior confiança
rules_franca[rules_franca['confidence'] == rules_franca['confidence'].max()]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
810,"(ALARM CLOCK BAKELIKE GREEN, MINI PAINT SET VI...",(ALARM CLOCK BAKELIKE PINK),0.021692,0.086768,0.021692,1.0,11.525000,0.019810,inf
872,"(RED RETROSPOT MINI CASES, ALARM CLOCK BAKELIK...",(ALARM CLOCK BAKELIKE RED),0.021692,0.080260,0.021692,1.0,12.459459,0.019951,inf
884,"(SPACEBOY LUNCH BOX, ALARM CLOCK BAKELIKE GREEN)",(ALARM CLOCK BAKELIKE RED),0.023861,0.080260,0.023861,1.0,12.459459,0.021946,inf
895,"(ALARM CLOCK BAKELIKE GREEN, PLASTERS IN TIN W...",(PLASTERS IN TIN SPACEBOY),0.028200,0.117137,0.028200,1.0,8.537037,0.024896,inf
902,"(ALARM CLOCK BAKELIKE ORANGE, ALARM CLOCK BAKE...",(ALARM CLOCK BAKELIKE RED),0.026030,0.080260,0.026030,1.0,12.459459,0.023941,inf
...,...,...,...,...,...,...,...,...,...
2440,"(SET/6 RED SPOTTY PAPER PLATES, PACK OF 20 SKU...",(PACK OF 6 SKULL PAPER PLATES),0.023861,0.047722,0.023861,1.0,20.954545,0.022722,inf
2441,"(SET/6 RED SPOTTY PAPER PLATES, PACK OF 20 SKU...",(SET/6 RED SPOTTY PAPER CUPS),0.023861,0.117137,0.023861,1.0,8.537037,0.021066,inf
2443,"(PACK OF 20 SKULL PAPER NAPKINS, SET/6 RED SPO...",(SET/6 RED SPOTTY PAPER PLATES),0.023861,0.108460,0.023861,1.0,9.220000,0.021273,inf
2449,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...","(SET/6 RED SPOTTY PAPER CUPS, PACK OF 6 SKULL ...",0.023861,0.034707,0.023861,1.0,28.812500,0.023033,inf


In [9]:
# c) Qual a regra com maior lift?

# usar o max para obter o maior lift
rules_franca[rules_franca['lift'] == rules_franca['lift'].max()]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1926,"(DOLLY GIRL CHILDRENS BOWL, SPACEBOY CHILDRENS...","(SPACEBOY CHILDRENS BOWL, DOLLY GIRL CHILDRENS...",0.023861,0.028200,0.023861,1.000000,35.461538,0.023188,inf
1931,"(SPACEBOY CHILDRENS BOWL, DOLLY GIRL CHILDRENS...","(DOLLY GIRL CHILDRENS BOWL, SPACEBOY CHILDRENS...",0.028200,0.023861,0.023861,0.846154,35.461538,0.023188,6.344902


In [10]:
# d) Quantas regras possuem suporte maior ou igual a 0.02 e confiança maior ou igual a 0.8?

rules_franca[(rules_franca['support'] >= 0.02) & (rules_franca['confidence'] >= 0.8)].size

4509

In [11]:
# e) Quantas regras possuem lift maior ou igual a 4 e confiança maior ou igual a 0.8?

rules_franca[(rules_franca['lift'] >= 4) & (rules_fraca['confidence'] >= 0.8)].size

4509

#### f) Quais produtos você recomendaria colocar próximos se você fosse organizar um supermercado físico?

- DOLLY GIRL CHILDRENS.. 
- SPACEBOY CHILDRENS BOWL.. 
- SET/6 RED SPOTTY PAPER PLATES 
- SET/6 RED SPOTTY PAPER CUPS

In [13]:
# 5) Faça o mesmo procedimento para a Alemanha (Country=’Germany’).
#Existe alguma similaridade nas principais regras? Quais produtos você recomendaria colocar próximos 
# se você fosse organizar um supermercado físico?

cesta_alemanha = (
    df[df['Country'] == 'Germany']
    .groupby(['InvoiceNo', 'Description'])['Quantity']
    .sum().unstack().fillna(0).reset_index()
    .set_index('InvoiceNo')
)
cesta_alemanha = cesta_franca.applymap(encode_units)
cesta_alemanha

# referência para o applymap: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.applymap.html

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C579532,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C579562,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C580161,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Maior suporte

# dúvida sobre o 0,02
freq_alemanha = apriori(cesta_alemanha, min_support = 0.02, use_colnames = True)
rules_alemanha = association_rules(freq_franca, metric = 'support', min_threshold = 0.02)
rules_alemanha [rules_alemanha ['support'] == rules_alemanha ['support'].max()]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
770,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.108460,0.117137,0.104121,0.960000,8.195556,0.091417,22.071584
771,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.117137,0.108460,0.104121,0.888889,8.195556,0.091417,8.023861


In [15]:
# Maior confiança

rules_alemanha [rules_alemanha ['confidence'] == rules_alemanha ['confidence'].max()]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
810,"(ALARM CLOCK BAKELIKE GREEN, MINI PAINT SET VI...",(ALARM CLOCK BAKELIKE PINK),0.021692,0.086768,0.021692,1.0,11.525000,0.019810,inf
872,"(RED RETROSPOT MINI CASES, ALARM CLOCK BAKELIK...",(ALARM CLOCK BAKELIKE RED),0.021692,0.080260,0.021692,1.0,12.459459,0.019951,inf
884,"(SPACEBOY LUNCH BOX, ALARM CLOCK BAKELIKE GREEN)",(ALARM CLOCK BAKELIKE RED),0.023861,0.080260,0.023861,1.0,12.459459,0.021946,inf
895,"(ALARM CLOCK BAKELIKE GREEN, PLASTERS IN TIN W...",(PLASTERS IN TIN SPACEBOY),0.028200,0.117137,0.028200,1.0,8.537037,0.024896,inf
902,"(ALARM CLOCK BAKELIKE ORANGE, ALARM CLOCK BAKE...",(ALARM CLOCK BAKELIKE RED),0.026030,0.080260,0.026030,1.0,12.459459,0.023941,inf
...,...,...,...,...,...,...,...,...,...
2440,"(SET/6 RED SPOTTY PAPER PLATES, PACK OF 20 SKU...",(PACK OF 6 SKULL PAPER PLATES),0.023861,0.047722,0.023861,1.0,20.954545,0.022722,inf
2441,"(SET/6 RED SPOTTY PAPER PLATES, PACK OF 20 SKU...",(SET/6 RED SPOTTY PAPER CUPS),0.023861,0.117137,0.023861,1.0,8.537037,0.021066,inf
2443,"(PACK OF 20 SKULL PAPER NAPKINS, SET/6 RED SPO...",(SET/6 RED SPOTTY PAPER PLATES),0.023861,0.108460,0.023861,1.0,9.220000,0.021273,inf
2449,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...","(SET/6 RED SPOTTY PAPER CUPS, PACK OF 6 SKULL ...",0.023861,0.034707,0.023861,1.0,28.812500,0.023033,inf


In [16]:
# Maior lift

rules_alemanha [rules_alemanha ['lift'] == rules_alemanha ['lift'].max()]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1926,"(DOLLY GIRL CHILDRENS BOWL, SPACEBOY CHILDRENS...","(SPACEBOY CHILDRENS BOWL, DOLLY GIRL CHILDRENS...",0.023861,0.028200,0.023861,1.000000,35.461538,0.023188,inf
1931,"(SPACEBOY CHILDRENS BOWL, DOLLY GIRL CHILDRENS...","(DOLLY GIRL CHILDRENS BOWL, SPACEBOY CHILDRENS...",0.028200,0.023861,0.023861,0.846154,35.461538,0.023188,6.344902


In [17]:
# Suporte maior ou igual a 0.02 e confiança maior ou igual a 0.8

rules_alemanha[(rules_alemanha['support'] >= 0.02) & (rules_alemanha['confidence'] >= 0.8)].size

4509

In [18]:
# Lift maior ou igual a 4 e confiança maior ou igual a 0.8

rules_alemanha[(rules_alemanha['lift'] >= 4) & (rules_alemanha['confidence'] >= 0.8)].size

4509

#### f) Quais produtos você recomendaria colocar próximos se você fosse organizar um supermercado físico?
- SET/6 RED SPOTTY PAPER PLATES
- SET/6 RED SPOTTY PAPER CUPS
- DOLLY GIRL CHILDRENS BOWL, SPACEBOY CHILDRENS...
- SPACEBOY CHILDRENS BOWL, DOLLY GIRL CHILDRENS...